In [1]:
%pwd

'c:\\Users\\jaysh\\OneDrive\\Desktop\\ML_Projects\\End-to-End_ML_Projects\\research'

In [2]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\jaysh\\OneDrive\\Desktop\\ML_Projects\\End-to-End_ML_Projects'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from red_wine_quality_prediction.constants import*

In [8]:
from red_wine_quality_prediction.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from red_wine_quality_prediction import logger
from red_wine_quality_prediction.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-13 12:36:11,796: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 12:36:11,802: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 12:36:11,811: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-13 12:36:11,815: INFO: common: created directory at: artifacts]
[2024-03-13 12:36:11,819: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-13 12:36:14,688: INFO: 3447005916: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4b28eda31554e671583b8984813309f1d95eb69e1055e048accf107a1cdbfa55"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 307A:1DD5CE:3B1954:4F1854:65F1508D
Accept-Ranges: bytes
Date: Wed, 13